# Day 5: ADF Test & Differencing

### Import and setup libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
import plotly.io as pio

pio.templates.default = "ggplot2" 

### Data imports and preprocessing

In [2]:
gold = pd.read_csv("../data/gold_prices.csv", parse_dates=["Date"], index_col="Date")

# Preprocess data to match with spot prices approximately
gold['Price'] = gold['Price'].astype(float)
gold['Price'] = gold['Price'] * 10.8
gold['Price'] = gold['Price'].round(0)

In [3]:
px.line(gold, x=gold.index, y="Price", title="Gold Prices (GLD) Over Time", labels={"Price": "Price (USD)"})

## 1. Augmented Dickey-Fuller (ADF) Test

The **Augmented Dickey-Fuller (ADF) test** is a statistical test for stationarity. It tests the null hypothesis that the time series has a unit root (i.e., it is non-stationary).

**Interpretation:**
- **p-value < 0.05**: Reject null hypothesis → Series is **stationary**
- **p-value ≥ 0.05**: Fail to reject null hypothesis → Series is **non-stationary**

A more negative ADF statistic provides stronger evidence against the null hypothesis of non-stationarity.

In [6]:
from statsmodels.tsa.stattools import adfuller

def adf_test(series, name=''):
    """
    Perform Augmented Dickey-Fuller test and print results
    """
    result = adfuller(series.dropna())
    
    print(f'--- ADF Test Results for {name} ---')
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value:.3f}')
    
    if result[1] <= 0.05:
        print(f"\\n✓ Result: The series IS stationary (p-value = {result[1]:.6f} < 0.05)")
    else:
        print(f"\\n✗ Result: The series IS NOT stationary (p-value = {result[1]:.6f} ≥ 0.05)")
    print()

## 2. Differencing to Achieve Stationarity

**Differencing** is the process of computing the change between consecutive observations:

```
Price_Diff(t) = Price(t) - Price(t-1)
```

This transformation removes trends and can make a non-stationary series stationary. Instead of modeling absolute prices, we model the *change* in price from one period to the next.

In [7]:
# Run ADF test on raw gold prices
adf_test(gold['Price'], 'Gold Price (Original)')

--- ADF Test Results for Gold Price (Original) ---
ADF Statistic: 4.657182
p-value: 1.000000
Critical Values:
   1%: -3.433
   5%: -2.863
   10%: -2.567
\n✗ Result: The series IS NOT stationary (p-value = 1.000000 ≥ 0.05)



## 3. Visual Comparison: Original vs Differenced

Let's visualize both the original price series and the differenced series side-by-side to see the transformation effect.

In [10]:
# Apply first-order differencing
gold['Price_Diff'] = gold['Price'].diff()

# Display results
print(gold[['Price', 'Price_Diff']].head(10))
print(f"Original Price - Mean: {gold['Price'].mean():.2f}, Std: {gold['Price'].std():.2f}")
print(f"Differenced Price - Mean: {gold['Price_Diff'].mean():.2f}, Std: {gold['Price_Diff'].std():.2f}")

             Price  Price_Diff
Date                          
2016-01-11  1131.0         NaN
2016-01-12  1125.0        -6.0
2016-01-13  1131.0         6.0
2016-01-14  1113.0       -18.0
2016-01-15  1124.0        11.0
2016-01-19  1123.0        -1.0
2016-01-20  1138.0        15.0
2016-01-21  1139.0         1.0
2016-01-22  1134.0        -5.0
2016-01-25  1146.0        12.0
Original Price - Mean: 1856.81, Std: 662.43
Differenced Price - Mean: 1.33, Std: 21.02


## 4. ADF Test on Differenced Data

Now let's run the ADF test again on the differenced series to confirm it's stationary.

In [11]:
# Create 2-panel comparison plot
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Original Gold Prices", "Differenced Gold Prices (Daily Change)"),
    vertical_spacing=0.12
)

# Original prices
fig.add_trace(
    go.Scatter(x=gold.index, y=gold['Price'], mode='lines', name='Original Price', line=dict(color='blue')),
    row=1, col=1
)

# Differenced prices
fig.add_trace(
    go.Scatter(x=gold.index, y=gold['Price_Diff'], mode='lines', name='Price Change', line=dict(color='red')),
    row=2, col=1
)

# Add horizontal line at y=0 for differenced plot
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

# Update layout
fig.update_layout(
    height=700,
    title_text="Comparison: Original vs Differenced Gold Prices",
    showlegend=False
)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
fig.update_yaxes(title_text="Price Change (USD)", row=2, col=1)

fig.show()

### Interpretation: Why Did the p-value Change?

**Original Gold Prices:**
- High p-value (likely > 0.05) indicates non-stationarity
- The series has a strong upward trend
- Mean increases over time
- ADF test fails to reject the null hypothesis of a unit root

**Differenced Gold Prices:**
- Very low p-value (likely < 0.01) indicates stationarity
- The differencing removed the trend component
- The series now fluctuates around a constant mean (near zero)
- ADF test strongly rejects the null hypothesis

**What This Means for Modeling:**

✓ **We can now use ARIMA models!** The differenced data satisfies the stationarity assumption required by these models.

The "I" in ARIMA stands for "Integrated," which refers to the number of differencing operations needed. Since we applied first-order differencing (d=1), we would use an ARIMA(p,1,q) model where:
- **p** = autoregressive order
- **d** = 1 (one differencing operation)
- **q** = moving average order

By transforming non-stationary data into stationary data through differencing, we've made the time series suitable for forecasting!

In [12]:
# Run ADF test on differenced prices
adf_test(gold['Price_Diff'], 'Gold Price (Differenced)')

--- ADF Test Results for Gold Price (Differenced) ---
ADF Statistic: -7.834985
p-value: 0.000000
Critical Values:
   1%: -3.433
   5%: -2.863
   10%: -2.567
\n✓ Result: The series IS stationary (p-value = 0.000000 < 0.05)

